In [70]:
import os

import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import torchvision
from torchvision import datasets
from torchvision.io import read_image
from torchvision.transforms import ToTensor
from torchvision.transforms import ToTensor

In [73]:
simple_transforms = torch.nn.Sequential(
    torchvision.transforms.Resize(size=(256, 256)),
    #transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
)

In [41]:
class CocoNoCropping(Dataset):
    def __init__(self, img_ids, my_annotations_file, img_dir,
                 transform=None, target_transform=None):

        # dataDir, dataType, annFile, 
        # self.coco_datatype = coco_datatype
        # self.coco_annotations_file = 
        
        with open(img_ids, 'r') as f:
            self.img_ids = json.loads(f.read()) # python list saved as txt IDs as ints
        with open(my_annotations_file, 'r') as f:
            self.ids_to_labels = json.load(f)   # python dictionary saved as json IDs as strings
            
        self.img_dir = img_dir
        
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_ids)

    def __getitem__(self, idx):
        img_id = str(self.img_ids[idx])
        img_name = (12-len(img_id)) * '0' + img_id + '.jpg'
        
        img_path = os.path.join(self.img_dir, img_name)
        image = read_image(img_path)
        
        label = self.ids_to_labels[img_id]
        if self.transform:
            image = self.transform(image)
            
        if self.target_transform:
            label = self.target_transform(label)
        sample = {"image": image, "label": label}
        return sample

In [42]:
%ls ../my_splits/

dev_ids.txt     test_ids.txt    train1_ids.txt  train_ids.txt


In [43]:
%ls ../my_annotations/

imgIds_to_labels_train2017.json     strCatNms_to_imgIds_train2017.json
imgIds_to_labels_val2017.json       strCatNms_to_imgIds_val2017.json


In [44]:
%ls ../images/

train2017/ val2017/


In [75]:
train1_dataset = CocoNoCropping(
    img_ids = '../my_splits/train1_ids.txt',
    my_annotations_file = '../my_annotations/imgIds_to_labels_train2017.json',
    img_dir = '../images/train2017/',
    transform=simple_transforms
)

In [84]:
train1_dataloader = DataLoader(train1_dataset, batch_size=8, shuffle=True)

In [85]:
next(iter(train1_dataloader))

{'image': tensor([[[[107, 114, 108,  ..., 107, 125, 152],
           [104, 106,  95,  ...,  85, 107, 140],
           [ 95,  98,  80,  ...,  73,  88, 124],
           ...,
           [128, 129, 129,  ..., 166, 189, 155],
           [132, 136, 132,  ..., 186, 193, 192],
           [131, 131, 131,  ..., 192, 188, 192]],
 
          [[123, 131, 125,  ..., 108, 119, 137],
           [123, 127, 109,  ...,  87, 102, 125],
           [117, 123,  93,  ...,  75,  86, 112],
           ...,
           [163, 166, 167,  ..., 178, 205, 158],
           [170, 172, 173,  ..., 179, 181, 175],
           [166, 162, 168,  ..., 179, 176, 175]],
 
          [[ 97,  99,  95,  ..., 100, 116, 141],
           [ 94,  90,  80,  ...,  77,  98, 128],
           [ 85,  83,  63,  ...,  63,  80, 113],
           ...,
           [142, 146, 146,  ..., 172, 197, 151],
           [152, 156, 155,  ..., 173, 173, 168],
           [152, 150, 154,  ..., 174, 167, 169]]],
 
 
         [[[ 93,  99,  93,  ...,  74,  73,  70],


In [86]:
for batch in train1_dataloader:
    inputs = batch['image'] # [batch size, channel, h, w]
    print(inputs.shape, inputs.dtype)
    
    labels = batch['label'] # list with num classes tensors, each tensor with batch-size labels
    print(labels) #labels.shape, labels.dtype
    
    print()
 
    #break

torch.Size([8, 3, 256, 256]) torch.uint8
[tensor([1, 0, 0, 1, 0, 0, 1, 0]), tensor([0, 0, 0, 0, 0, 0, 0, 0]), tensor([0, 1, 1, 0, 0, 0, 1, 0]), tensor([0, 1, 0, 1, 1, 1, 1, 1])]

torch.Size([8, 3, 256, 256]) torch.uint8
[tensor([0, 0, 0, 0, 0, 0, 0, 0]), tensor([1, 1, 1, 0, 0, 1, 0, 1]), tensor([0, 0, 0, 1, 0, 0, 0, 0]), tensor([0, 0, 0, 1, 0, 0, 0, 0])]

torch.Size([8, 3, 256, 256]) torch.uint8
[tensor([0, 0, 1, 0, 1, 0, 0, 0]), tensor([0, 0, 0, 0, 0, 1, 1, 0]), tensor([0, 0, 0, 1, 0, 1, 0, 0]), tensor([0, 0, 0, 1, 0, 0, 0, 0])]

torch.Size([8, 3, 256, 256]) torch.uint8
[tensor([0, 0, 0, 0, 0, 0, 0, 0]), tensor([0, 0, 0, 0, 0, 0, 0, 0]), tensor([1, 0, 0, 0, 1, 0, 0, 0]), tensor([0, 0, 0, 0, 0, 1, 1, 0])]

torch.Size([8, 3, 256, 256]) torch.uint8
[tensor([0, 0, 1, 1, 1, 0, 0, 0]), tensor([1, 0, 0, 0, 0, 0, 1, 0]), tensor([0, 0, 0, 0, 0, 1, 0, 1]), tensor([0, 1, 0, 1, 0, 1, 1, 1])]

torch.Size([8, 3, 256, 256]) torch.uint8
[tensor([0, 0, 1, 0, 0, 0, 0, 0]), tensor([0, 0, 0, 1, 0, 0, 1, 